# Machine Learning in Active Magnetic Regenerators

This Notebook presents the results for the application of several Machine Learning techniques in the prediction of the Cooling Capacity ($\dot{Q}_\mathrm{C}$ or $Qc$) and the Magnetization Power ($\dot{W}_\mathrm{m}$ or $Wm$) of Active Magnetic Regenerators. The Dataset consists in 532 points coming from numerical simulations. The Machine Learning techniques which were evaluated are:

- Ridge Regression;
- Ridge Regression with Second Degree Polynomial Features;
- Ridge Regression with Third Degree Polynomial Features;
- Ridge Regression with Forth Degree Polynomial Features;
- K-Nearest Neighbors (KNN);
- Random Forest (RF);
- Support Vector Regression (SVR);
- Extreme Gradient Boosting (XGBoost);

### Importing the Modules

In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures as Poly
from sklearn.pipeline import make_pipeline
from sklearn.metrics import r2_score
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import Ridge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from xgboost.sklearn import XGBRegressor

import joblib

mpl.style.use('default')
plt.rc('text',usetex = True)
plt.rc('font', family='serif',size = 12)

C:\Users\Guilherme\Anaconda3\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


### Plotting Functions 

In [2]:
def plot_results(y_train,y_pred_train,y_test,y_pred_test, Method,Obj):
    
    plt.figure()
    plt.plot(y_train,y_pred_train,'o',color = '#1f77b4', mfc='none', label = 'Training Set')
    plt.plot(y_test,y_pred_test,'x' ,color = '#2ca02c', mfc='none',label = 'Test Set',)
    
    if Obj == 'Qc':
        Aux = np.linspace(10,300)
        plt.plot(Aux,0.9*Aux,'k--')
        plt.plot(Aux,1.1*Aux,'k--')
        plt.text(140, 180, '$+ 10 \%$', fontsize=12)
        plt.text(150, 120, '$- 10 \%$', fontsize=12)
        plt.legend()
        plt.ylabel(r' $\dot{Q}_\mathrm{C}$ [W] - ' + Method )
        plt.xlabel(r' $\dot{Q}_\mathrm{C}$ [W] - Target')
        plt.grid(linestyle='dotted')
        plt.savefig('Plots/Qc/Qc - '+Method +'.png', format = 'png', bbox_inches='tight') 
    
    elif Obj == 'Wm':
        Aux = np.linspace(10,60)
        plt.plot(Aux,0.9*Aux,'k--')
        plt.plot(Aux,1.1*Aux,'k--')
        plt.text(23, 17, '$- 10 \%$', fontsize=12)
        plt.text(19, 28, '$+ 10 \%$', fontsize=12)
        plt.legend()
        plt.ylabel(r' $\dot{W}_\mathrm{AMR,Mag}$ [W] - ' + Method )
        plt.xlabel(r' $\dot{W}_\mathrm{AMR,Mag}$ [W] - Target')
        plt.grid(linestyle='dotted')
        plt.savefig('Plots/Wm/Wm - '+Method +'.png', format = 'png', bbox_inches='tight') 

### Reading the Input Parameteres 

The independent variable are the Width ($H$), Height ($H$) and Length ($L$) of the porous medium, the frequency ($f$) and mass flow rate ($mf$) of the Active Magnetic Regenerator, the Inlet Temperatures at the Hot ($Th$) and Cold ($Tc$) ends and the Applied Magnetic Field ($B$).

The dependent variables are the Cooling Capacity ($Qc$) and Magnetization Power ($Wm$).

In [3]:
Inputs = pd.read_excel('Data/Data_AMR.xlsx',index_col = 0) # Input Data
X = Inputs[['W','H','f','mf','B','L','Th','Tc']]
y = Inputs[['Qc','Wm']]

### Train-Test Split

The Dataset is divided into Training (80%) and Test (20%) sets.

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y,
                                                    test_size = 0.2,
                                                    random_state = 42)  

### Training Function

The models are trained by a Grid Search cross validation technique, with the exception of the Ridge Regressions. 

In [5]:
def ML_Model(X_train,y_train,X_test,y_test,Algo,Obj):
    
    # Ridge Regression
    if Algo == 'Ridge1':
        model = make_pipeline(Poly(degree=1, interaction_only = False),Ridge(alpha=0.5))
        param = {}
        
    # Ridge Regression with Second Degree Polynomial Features    
    elif Algo == 'Ridge2':
        model = make_pipeline(Poly(degree=2, interaction_only = False),Ridge(alpha=0.5))
        param = {}
        
    # Ridge Regression with Third Deree Polynomial Features 
    elif Algo == 'Ridge3':
        model = make_pipeline(Poly(degree=3, interaction_only = False),Ridge(alpha=0.5))
        param = {}
        
    # Ridge Regression with Forth Degree Polynomial Features     
    elif Algo == 'Ridge4':
        model = make_pipeline(Poly(degree=4, interaction_only = False),Ridge(alpha=0.5))
        param = {}
    
    # K-Nearest Neighbors
    elif Algo == 'KNN':
        param = {'n_neighbors': [1,5,10,20,50,100],
                 'leaf_size': [1,5,10,20,50,100],
                 'p': [1,3,5],
                 'weights': ('uniform', 'distance'),
                }
        model = KNeighborsRegressor(algorithm='auto')
    
    # Ranfom Forest
    elif Algo == 'RF':
        param = {'n_estimators': [5,25,50,100],
                 'max_features': ['auto', 'sqrt'],
                 'max_depth': [5,10,50],
                 'min_samples_split': [2,6,10],
                 'min_samples_leaf' : [1,3,5]
                }
        model = RandomForestRegressor(random_state=42)
    
    #Support Vector Regression
    elif Algo =='SVR':
        param = {'kernel':['linear','rbf', 'poly', 'sigmoid'],
                 #'C':[0.1,1, 10, 100],
                 #'gamma': [1,0.1,0.01,0.001]
                 #'epsilon':[0.1, 0.2]
                  }
        model = SVR()
    
    #XGBoost 
    elif Algo == 'XGBoost':
        param = {'objective':['reg:squarederror'],
                 'learning_rate': [0.01, 0.02],
                 'max_depth': [5, 6, 7],
                 'min_child_weight': [1,3],
                 'alpha' :[1,5],
                 'gamma': [1,2],
                 'subsample': [0.5, 0.75],
                 'colsample_bytree': [0.75,1],
                 'n_estimators': [500, 1000]
                }
        model = XGBRegressor()
    
    #GridSearch Cross Validation
    model = GridSearchCV(model,
                         param_grid=param,
                         scoring = 'r2',
                         verbose = 2
                        )
    
    #Training the Model
    model.fit(X_train, y_train[Obj])

    #R2 for the Training Set
    print('The coefficient of determination for '+ Obj +' Training Set using '+ Algo +' is '+
          str(r2_score(y_train[Obj],model.predict(X_train).T)))

    # R2 for the Test Set
    print('The coefficient of determination for '+ Obj +' Test Set using '+ Algo +' is '+
          str(r2_score(y_test[Obj], model.predict(X_test).T))) 

    # Plotting the Results
    plot_results(y_train[Obj], model.predict(X_train).T, y_test[Obj],model.predict(X_test).T, Algo, Obj) 
  
    # Saving the Model
    joblib.dump(model, 'Models/'+Obj+'/'+Obj+' - '+Algo+'.pkl')
    
    #Returning the Train and Test Sets R2
    return r2_score(y_train[Obj],model.predict(X_train).T),r2_score(y_test[Obj], model.predict(X_test).T)

### Evaluation of the Techniques

In [7]:
#List of Algorithms being Evaluated
Algo = ['Ridge1','Ridge2','Ridge3','Ridge4','KNN','RF','SVR','XGBoost']

In [ ]:
# Lists for saving the R2 Coefficients
R2_Qc = []
R2_Wm = []

#Running the ML_Model functions for each Algorithm and Objective
for i in Algo:
        R2_Qc.append(ML_Model(X_train,y_train,X_test,y_test,str(i),'Qc'))
        R2_Wm.append(ML_Model(X_train,y_train,X_test,y_test,str(i),'Wm'))

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END .................................................... total time=   0.0s
[CV] END .................................................... total time=   0.0s
[CV] END .................................................... total time=   0.0s
[CV] END .................................................... total time=   0.0s
[CV] END .................................................... total time=   0.0s
The coefficient of determination for Qc Training Set using Ridge1 is 0.8885910303422555
The coefficient of determination for Qc Test Set using Ridge1 is 0.877878706472577
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END .................................................... total time=   0.0s
[CV] END .................................................... total time=   0.0s
[CV] END .................................................... total time=   0.0s
[CV] END .................................................... t

C:\Users\Guilherme\Anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=1.52909e-20): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
C:\Users\Guilherme\Anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=1.43943e-20): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
C:\Users\Guilherme\Anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=1.55404e-20): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
C:\Users\Guilherme\Anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=1.45742e-20): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
C:\Users\Guilherme\Anaconda3\lib\site-packages\sklearn\linear_model\

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END .................................................... total time=   0.0s
[CV] END .................................................... total time=   0.0s
[CV] END .................................................... total time=   0.0s
[CV] END .................................................... total time=   0.0s
[CV] END .................................................... total time=   0.0s
The coefficient of determination for Wm Training Set using Ridge3 is 0.999799987136918
The coefficient of determination for Wm Test Set using Ridge3 is 0.9995876064444003


C:\Users\Guilherme\Anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=1.52909e-20): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
C:\Users\Guilherme\Anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=1.43943e-20): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
C:\Users\Guilherme\Anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=1.55404e-20): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
C:\Users\Guilherme\Anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=1.45742e-20): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
C:\Users\Guilherme\Anaconda3\lib\site-packages\sklearn\linear_model\

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END .................................................... total time=   0.0s
[CV] END .................................................... total time=   0.0s
[CV] END .................................................... total time=   0.0s
[CV] END .................................................... total time=   0.0s
[CV] END .................................................... total time=   0.0s
The coefficient of determination for Qc Training Set using Ridge4 is 0.9881337394028719
The coefficient of determination for Qc Test Set using Ridge4 is 0.9794580562597117


...

C:\Users\Guilherme\Anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


[CV] END alpha=5, colsample_bytree=1, gamma=1, learning_rate=0.01, max_depth=7, min_child_weight=3, n_estimators=500, objective=reg:squarederror, subsample=0.5; total time=   0.4s


C:\Users\Guilherme\Anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


[CV] END alpha=5, colsample_bytree=1, gamma=1, learning_rate=0.01, max_depth=7, min_child_weight=3, n_estimators=500, objective=reg:squarederror, subsample=0.5; total time=   0.3s


C:\Users\Guilherme\Anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


In [ ]:
#Saving the R2 coefficients of the Training and Test Sets for the Qc and Wm
pd.DataFrame(R2_Qc, columns = ['Train','Test'], index = Algo).to_excel('R2_Qc.xlsx')
pd.DataFrame(R2_Wm, columns = ['Train','Test'], index = Algo).to_excel('R2_Wm.xlsx')